# # Dataset-4 Phishing Email

# Exploration

In [3]:
import numpy as np
import pandas as pd
from scipy import stats as sps
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Latex

file_path = '/Users/dakshkhetarpaul/Desktop/ML443-Final-Project/Datasets/Phishing_Email.csv'

# Load the data
data = pd.read_csv(file_path)

# Number of samples and raw features
num_samples = data.shape[0]
num_features = data.shape[1]
print(f"Number of samples: {num_samples}")
print(f"Number of raw features: {num_features}")

Number of samples: 18650
Number of raw features: 3
